In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sb
import matplotlib.pyplot as plt
from datetime import datetime
import math
from IPython.display import display, HTML
from datetime import date
import multiprocessing

import os

In [ ]:
my_bucket = os.getenv('WORKSPACE_BUCKET')
CDR_version=os.getenv("WORKSPACE_CDR")

### Curate Medication

In [ ]:
# This query represents dataset "MDD_nopsychremoved_drugs" for domain "drug" and was generated for All of Us Registered Tier Dataset v7
import pandas
import os

dataset_78105840_drug_sql = """
    SELECT
        d_exposure.person_id,
        d_standard_concept.concept_name as standard_concept_name,
        d_exposure.drug_exposure_start_datetime,
        d_exposure.refills,
        d_exposure.quantity,
        d_exposure.dose_unit_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.drug_exposure` d_exposure 
        WHERE
            (
                drug_concept_id IN  (
                    SELECT
                        DISTINCT ca.descendant_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria_ancestor` ca 
                    JOIN
                        (
                            select
                                distinct c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (
                                    select
                                        cast(cr.id as string) as id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                    WHERE
                                        concept_id IN (
                                            19080226, 19136626, 35603277, 40234834, 43560354, 44507700, 46275300, 703470, 703547, 705755, 710062, 712615, 713109, 714684, 715259, 715939, 716968, 717607, 721724, 722031, 725131, 733896, 735979, 738156, 739138, 743670, 750982, 751412, 754270, 755695, 757688, 766209, 766814, 778268, 781705, 785788, 794147, 797617, 798834, 800878
                                        ) 
                                        AND full_text LIKE '%_rank1]%'
                                ) a 
                                    ON (
                                        c.path LIKE CONCAT('%.',
                                    a.id,
                                    '.%') 
                                    OR c.path LIKE CONCAT('%.',
                                    a.id) 
                                    OR c.path LIKE CONCAT(a.id,
                                    '.%') 
                                    OR c.path = a.id) 
                                WHERE
                                    is_standard = 1 
                                    AND is_selectable = 1
                                ) b 
                                    ON (
                                        ca.ancestor_id = b.concept_id
                                    )
                            )
                        )  
                        AND (
                            d_exposure.PERSON_ID IN (
                                SELECT
                                    distinct person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (4031328, 4094358, 4262111, 4037669, 35615155, 4152280, 4282316, 35615153, 35615152, 4001733, 4324945, 4298317) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 1 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 1 
                                                )
                                            ) criteria 
                                        ) ))
                            ) d_exposure 
                        LEFT JOIN
                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_standard_concept 
                                ON d_exposure.drug_concept_id = d_standard_concept.concept_id"""

medication_df = pandas.read_gbq(
    dataset_78105840_drug_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

In [ ]:
import pandas as pd
import numpy as np 
import datetime

In [ ]:
medication_df['drug'] = medication_df.standard_concept_name.apply(lambda x: x.split(' ', 1)[0])
medication_df['drug'] = medication_df['drug'].str.replace('\d+', '')
medication_df = medication_df[~medication_df['drug'].str.contains(r'\d')]
medication_df = medication_df[~medication_df['drug'].isin(["PMDD" , "Sensor", "Sprinkle", "Smoking", "sulpiride"])]
medication_df['drug'] = medication_df['drug'].str.lower()

medication_df['category'] = (np.select([medication_df['drug'].isin(['citalopram', 'escitalopram', 'fluoxetine', 'fluvoxamine', 'paroxetine', 'sertraline']),
                                       medication_df['drug'].isin(['desvenlafaxine', 'duloxetine', 'levomilnacipran', 'milnacipran', 'venlafaxine']),
                                       medication_df['drug'].isin(['bupropion', 'mirtazapine']),
                                       medication_df['drug'].isin(['nefazodone', 'trazodone', 'vilazodone', 'vortioxetine']),
                                       medication_df['drug'].isin(['imipramine', 'amitriptyline', 'amoxapine', 
                                                                   'clomipramine', 'desipramine', 'doxepin',
                                                                   'maprotiline', 'nortriptyline', 'protriptyline','trimipramine']),
                                       medication_df['drug'].isin(['isocarboxazid', 'phenelzine', 'selegiline', 'tranylcypromine']),
                                       medication_df['drug'].isin(['sulrpide', 'aripiprazole', 'quetiapine', 'olanzapine',
                                                                   'clozapine', 'ziprasidone', 'risperidone'])]
                                      ,['SSRI','SNRI','Atypical','Serotonin','Tricyclic','MAOIs','Antipsychotics']))
# get the ids
MDDpatientIDs = medication_df.drop_duplicates(subset=['person_id'])['person_id']

#remove individuals who has duplicated GenericName and OrderStartDate 
MDD_med_dupsRemoved = medication_df[~(medication_df.duplicated(subset=['person_id', 'drug_exposure_start_datetime','category']))]
MDD_med_dupsRemoved = MDD_med_dupsRemoved.dropna(subset=['person_id'])

### Treatment Resistant Depression

In [ ]:
group_medUse = MDD_med_dupsRemoved.groupby(['person_id', 
                                                 'drug_exposure_start_datetime'])['drug'].apply(list).apply(lambda drug: ",".join(drug)) 

medInfo_MDDpatients = {}
for patient in MDDpatientIDs:
    medInfo_MDDpatients[patient] = pd.DataFrame(group_medUse[patient]).reset_index()
    #if you have cell where the values are the same i.e. SNRI, SNRI then remove the duplicate so its just SNRI
    medInfo_MDDpatients[patient]['drug'] = medInfo_MDDpatients[patient]['drug'].apply(lambda s: ','.join(set(s.split(','))))
treatment_resistant = medInfo_MDDpatients.copy()

In [ ]:
#calculating the adherence score for each patient
adherence_score_for_patient = {}
for patient in list(treatment_resistant.keys()):
    treatment_resistant[patient]["diff"] = treatment_resistant[patient]["drug_exposure_start_datetime"].diff().shift(-1)
    total_prescription_interval = len(treatment_resistant[patient]["drug"]) - 1
    
    times_lessthan_maxGap = (treatment_resistant[patient]["diff"] <= pd.Timedelta("98 days")).sum()
    adherence_score_for_patient[patient] = times_lessthan_maxGap/total_prescription_interval
    
# getting all the patients that pass adherence threshold of 0.75
pass_adherence_score = {}
for k, v in list(adherence_score_for_patient.items()):
    if v > 0.75:
         pass_adherence_score[k] = v

In [ ]:
for patient in list(treatment_resistant.keys()):
    #making a column to keep track of the minimal length of each drug 
    treatment_resistant[patient]["diff"] = treatment_resistant[patient]["drug_exposure_start_datetime"].diff().shift(-1)
    #make the last drug a lenght of 60 days so that it doesn't get removed
    treatment_resistant[patient]["diff"].iloc[-1] = pd.Timedelta("60 days")
    treatment_resistant[patient][["no_meets_min_length", "no_meets_max_length"]] = None
    
    for i in range(len(treatment_resistant[patient])):
        #if the minimal length is more than or equal to 6 weeks than it meets criteria and we add a 1
        if (treatment_resistant[patient]['diff'].loc[i] >= pd.Timedelta("42 days")):
            treatment_resistant[patient]['no_meets_min_length'].loc[i] = 0
        else:
            treatment_resistant[patient]['no_meets_min_length'].loc[i] = 1
    
for patient in list(treatment_resistant.keys()):
    for i in range(len(treatment_resistant[patient])-1):
        if ((treatment_resistant[patient]['drug'].loc[i] != 
            treatment_resistant[patient]['drug'].loc[i+1]) & 
            (treatment_resistant[patient]['diff'].loc[i] >= pd.Timedelta("98 days"))):
            treatment_resistant[patient]['no_meets_max_length'].loc[i] = 1
        else:
            treatment_resistant[patient]['no_meets_max_length'].loc[i] = 0

In [ ]:
#remove the drugs that are not at least 6 weeks and at most 14 weeks 
for patient in list(treatment_resistant.keys()):
    treatment_resistant[patient] = treatment_resistant[patient].loc[(treatment_resistant[patient]['no_meets_min_length'] != 1)]
    treatment_resistant[patient] = treatment_resistant[patient].loc[(treatment_resistant[patient]['no_meets_max_length'] != 1)]

In [ ]:
# Determining Patients who 180 days = 6 months
for patient in list(treatment_resistant.keys()):   
    #remove the patients that only took one medication class
    if (((treatment_resistant[patient]['drug_exposure_start_datetime'].iloc[-1] - 
        treatment_resistant[patient]['drug_exposure_start_datetime'].iloc[0]) < datetime.timedelta(days=180)) == True):
        del treatment_resistant[patient]

In [ ]:
TRD = treatment_resistant.copy()
for patient in list(TRD.keys()):
    if len(set(TRD[patient]['drug'])) < 3:
        del TRD[patient]
final_TRD_patients = set(list(pass_adherence_score.keys())) & set(list(TRD.keys()))
for patient in list(TRD.keys()):
    if patient not in final_TRD_patients:
        del TRD[patient]

In [ ]:
treat_res = pd.DataFrame({'PatientID': TRD.keys()})
treat_res.to_csv('./TRD.csv')

### Treatment Nonresponders

In [ ]:
group_medUse = MDD_med_dupsRemoved.groupby(['person_id', 'drug_exposure_start_datetime'])['category'].apply(list).apply(lambda category: ",".join(category))  

medInfo_MDDpatients = {}
for patient in MDDpatientIDs:
    medInfo_MDDpatients[patient] = pd.DataFrame(group_medUse[patient]).reset_index()
    #if you have cell where the values are the same i.e. SNRI, SNRI then remove the duplicate so its just SNRI
    medInfo_MDDpatients[patient]['category'] = medInfo_MDDpatients[patient]['category'].apply(lambda s: ','.join(set(s.split(','))))

In [ ]:
nonresponder = medInfo_MDDpatients.copy()

In [ ]:
for patient in list(nonresponder.keys()):
    #making a column to keep track of the minimal length of each drug 
    nonresponder[patient]["diff"] = nonresponder[patient]["drug_exposure_start_datetime"].diff().shift(-1)
    #make the last drug a lenght of 60 days so that it doesn't get removed
    nonresponder[patient]["diff"].iloc[-1] = pd.Timedelta("60 days")
    nonresponder[patient][["no_meets_min_length", "no_meets_max_length"]] = None
    
    for i in range(len(nonresponder[patient])):
        #if the minimal length is more than or equal to 6 weeks than it meets criteria and we add a 1
        if (nonresponder[patient]['diff'].loc[i] >= pd.Timedelta("42 days")):
            nonresponder[patient]['no_meets_min_length'].loc[i] = 0
        else:
            nonresponder[patient]['no_meets_min_length'].loc[i] = 1
    
for patient in list(nonresponder.keys()):
    for i in range(len(nonresponder[patient])-1):
        if ((nonresponder[patient]['category'].loc[i] != 
            nonresponder[patient]['category'].loc[i+1]) & 
            (nonresponder[patient]['diff'].loc[i] >= pd.Timedelta("98 days"))):
            nonresponder[patient]['no_meets_max_length'].loc[i] = 1
        else:
            nonresponder[patient]['no_meets_max_length'].loc[i] = 0

In [ ]:
# Step 3: Keep patients that have 1 or more medication classes i.e. switches
for patient in list(nonresponder.keys()):   
    #remove the patients that only took one medication class
    if (len(nonresponder[patient]['category'].unique()) < 1):
        del nonresponder[patient]

# Step 4: Remove patients that don't have a record length greater than or equal to 6 months 
for patient in list(nonresponder.keys()):   
    if ((nonresponder[patient]['drug_exposure_start_datetime'].iloc[-1] - 
        nonresponder[patient]['drug_exposure_start_datetime'].iloc[0]) < datetime.timedelta(days=180) == True):
        del nonresponder[patient]
        
# Step 5: Remove if long term user of most recent medication class
medInfo_2 = nonresponder.copy()

treatment_nonresp_df = pd.DataFrame()
for patient in list(medInfo_2.keys()): 
    count = 0
    for medclass in medInfo_2[patient]["category"][::-1]:
        if medInfo_2[patient]["category"].iloc[-1] == medclass:
            count += 1
        else: 
            break 
    treatment_nonresp_df = pd.concat([treatment_nonresp_df, pd.DataFrame([{'PatientId':patient, 
                                'Class': medInfo_2[patient]["category"].iloc[-1],
                                'Count': count}])],
                                     ignore_index = True)    
treatment_nonresp_df = treatment_nonresp_df[treatment_nonresp_df['Count'] > 3]
treatment_nonresp_df.to_csv('./nonresponders.csv')

In [ ]:
ssri_nonresponders = treatment_nonresp[treatment_nonresp['Class'] == 'SSRI']
snri_nonresponders = treatment_nonresp[treatment_nonresp['Class'] == 'SNRI']
tricyclic_nonresponders = treatment_nonresp[treatment_nonresp['Class'] == 'Tricyclic']
atypical_nonresponders = treatment_nonresp[treatment_nonresp['Class'] == 'Atypical']
serotonin_nonresponders = treatment_nonresp[treatment_nonresp['Class'] == 'Serotonin']

In [ ]:
ssri_nonresponders.to_csv('./ssri_nonresponders.csv')
snri_nonresponders.to_csv('./snri_nonresponders.csv')
tricyclic_nonresponders.to_csv('./tricyclic_nonresponders.csv')
atypical_nonresponders.to_csv('./atypical_nonresponders.csv')
serotonin_nonresponders.to_csv('./serotonin_nonresponders.csv')

### Treatment Responder

In [ ]:
group_medUse = MDD_med_dupsRemoved.groupby(['person_id', 'drug_exposure_start_datetime'])['category'].apply(list).apply(lambda category: ",".join(category))  

medInfo_MDDpatients = {}
for patient in MDDpatientIDs:
    medInfo_MDDpatients[patient] = pd.DataFrame(group_medUse[patient]).reset_index()
    #if you have cell where the values are the same i.e. SNRI, SNRI then remove the duplicate so its just SNRI
    medInfo_MDDpatients[patient]['category'] = medInfo_MDDpatients[patient]['category'].apply(lambda s: ','.join(set(s.split(','))))

In [ ]:
#6 month of medication history
medInfo_1 = medInfo_MDDpatients.copy()
# Determining Patients who only used 1 medication class the whole time 
for patient in list(medInfo_1.keys()):   
    #remove the patients that only took one medication class
    if (((medInfo_1[patient]['drug_exposure_start_datetime'].iloc[-1] - 
        medInfo_1[patient]['drug_exposure_start_datetime'].iloc[0]) < datetime.timedelta(days=180)) == True) or (medInfo_1[patient].shape[0] == 1):
        del medInfo_1[patient]
        
single_class_user = medInfo_1.copy() #six_month_ofData
for patient in list(medInfo_1.keys()):
    if (len(single_class_user[patient]['category'].unique()) != 1):
        del single_class_user[patient]
        
single_class_user_df = pd.DataFrame()
for patient in list(single_class_user.keys()):
    single_class_user_df = pd.concat([single_class_user_df, 
                                      pd.DataFrame([{'PatientId':patient, 
                                                     'Class': single_class_user[patient]["category"].iloc[-1]}])],
                                     ignore_index = True)
single_class_user_df_2 = single_class_user_df[~single_class_user_df['Class'].str.contains(',')].reset_index(drop=True)

responders = single_class_user_df_2.rename(columns={'PatientId': 'research_id'})
responders['research_id'] = responders['research_id'].astype('int')
responders = responders[~responders['Class'].isin(['0', 'Antipsychotics'])]
responders.to_csv('./responders.csv')

In [ ]:
ssri_responders = responders[responders['Class'] == 'SSRI']
snri_responders = responders[responders['Class'] == 'SNRI']
tricyclic_responders = responders[responders['Class'] == 'Tricyclic']
atypical_responders = responders[responders['Class'] == 'Atypical']
serotonin_responders = responders[responders['Class'] == 'Serotonin']

In [ ]:
ssri_responders.to_csv('./ssri_responders.csv')
snri_responders.to_csv('./snri_responders.csv')
tricyclic_responders.to_csv('./tricyclic_responders.csv')
atypical_responders.to_csv('./atypical_responders.csv')
serotonin_responders.to_csv('./serotonin_responders.csv')